# 0. Data Load

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/ratings_small.csv
/kaggle/input/the-movies-dataset/links.csv


In [2]:
from subprocess import check_output
import subprocess

files = check_output(['ls','../input/the-movies-dataset']).decode('utf8')
# files
for file in files.split("\n"):
#     print(file)
    path='../input/the-movies-dataset/'+ file
    popenobj=subprocess.Popen(['wc', '-l', path], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#     print(popenobj)
    result,error= popenobj.communicate()
    print("The file :",file,"has :",result.strip().split()[0],"rows")

The file : credits.csv has : b'45477' rows
The file : keywords.csv has : b'46420' rows
The file : links.csv has : b'45844' rows
The file : links_small.csv has : b'9126' rows
The file : movies_metadata.csv has : b'45572' rows
The file : ratings.csv has : b'26024290' rows
The file : ratings_small.csv has : b'100005' rows
The file :  has : b'0' rows


In [3]:
ratings_pd = pd.read_csv('/kaggle/input/the-movies-dataset/ratings.csv')
links_small_pd = pd.read_csv('/kaggle/input/the-movies-dataset/links_small.csv')
credits_pd = pd.read_csv('/kaggle/input/the-movies-dataset/credits.csv')
keywords_pd = pd.read_csv('/kaggle/input/the-movies-dataset/keywords.csv')
movies_metadata_pd = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')
ratings_small_pd = pd.read_csv('/kaggle/input/the-movies-dataset/ratings_small.csv')
links_pd = pd.read_csv('/kaggle/input/the-movies-dataset/links.csv')

print('*** df 생성 완료 ***')

*** df 생성 완료 ***


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 1. Memory Optimization

In [4]:
def check_dtypes(file_path):
    
    tmp = pd.read_csv(file_path, nrows=0)
    
    col_dtypes = {}
    for col in tmp.columns:
        df = pd.read_csv(file_path, usecols=[col])
        dtype = df[col].dtype
        
        if dtype == 'int' or dtype == 'float':
            c_min = df[col].min()
            c_max = df[col].max()
        elif dtype == 'object':
            n_unique = df[col].nunique()
            threshold = n_unique / df.shape[0]
            
        if dtype == 'int':
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                col_dtype = 'int8'
            elif c_min > np.iinfo(np.uint8).min and c_max < np.iinfo(np.uint8).max:
                col_dtype = 'uint8'
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                col_dtype = 'int16'
            elif c_min > np.iinfo(np.uint16).min and c_max < np.iinfo(np.uint16).max:
                col_dtype = 'uint16'
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                col_dtype = 'int32'
            elif c_min > np.iinfo(np.uint32).min and c_max < np.iinfo(np.uint32).max:
                col_dtype = 'uint32'
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                col_dtype = 'int64'
            elif c_min > np.iinfo(np.uint64).min and c_max < np.iinfo(np.uint64).max:
                col_dtype = 'uint64'
        elif dtype == 'float':
            if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                col_dtype = 'float32'
            else :
                col_dtype = 'float64'
        elif dtype == 'object':
            if threshold > 0.7:
                col_dtype = 'object'
            else :
                col_dtype = 'category'
                
        col_dtypes[col] = col_dtype
    
    return col_dtypes

In [5]:
data_types=check_dtypes('/kaggle/input/the-movies-dataset/movies_metadata.csv')
print("***movies_metadata***")
memory = movies_metadata_pd.memory_usage(index=True).sum()
print("Original dataset uses ", memory/ 1024**2," MB")
movies_metadata_pd = movies_metadata_pd.astype(data_types)
new_memory = movies_metadata_pd.memory_usage(index=True).sum()
print("dataset uses ",new_memory/ 1024**2," MB after changes")
print("memory saved =",(memory-new_memory)/ 1024**2," MB")
print("==========================================================")

data_types=check_dtypes('/kaggle/input/the-movies-dataset/credits.csv')
print("***credits***")
memory = credits_pd.memory_usage(index=True).sum()
print("Original dataset uses ", memory/ 1024**2," MB")
credits_pd = credits_pd.astype(data_types)
new_memory = credits_pd.memory_usage(index=True).sum()
print("dataset uses ",new_memory/ 1024**2," MB after changes")
print("memory saved =",(memory-new_memory)/ 1024**2," MB")
print("==========================================================")

data_types=check_dtypes('/kaggle/input/the-movies-dataset/keywords.csv')
print("***keywords***")
memory = keywords_pd.memory_usage(index=True).sum()
print("Original dataset uses ", memory/ 1024**2," MB")
keywords_pd = keywords_pd.astype(data_types)
new_memory = keywords_pd.memory_usage(index=True).sum()
print("dataset uses ",new_memory/ 1024**2," MB after changes")
print("memory saved =",(memory-new_memory)/ 1024**2," MB")
print("==========================================================")

data_types=check_dtypes('/kaggle/input/the-movies-dataset/links.csv')
print("***links***")
memory = links_pd.memory_usage(index=True).sum()
print("Original dataset uses ", memory/ 1024**2," MB")
links_pd = links_pd.astype(data_types)
new_memory = links_pd.memory_usage(index=True).sum()
print("dataset uses ",new_memory/ 1024**2," MB after changes")
print("memory saved =",(memory-new_memory)/ 1024**2," MB")
print("==========================================================")

data_types=check_dtypes('/kaggle/input/the-movies-dataset/ratings.csv')
print("***ratings***")
memory = ratings_pd.memory_usage(index=True).sum()
print("Original dataset uses ", memory/ 1024**2," MB")
ratings_pd = ratings_pd.astype(data_types)
new_memory = ratings_pd.memory_usage(index=True).sum()
print("dataset uses ",new_memory/ 1024**2," MB after changes")
print("memory saved =",(memory-new_memory)/ 1024**2," MB")


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


***movies_metadata***
Original dataset uses  8.3251953125  MB
dataset uses  6.794769287109375  MB after changes
memory saved = 1.530426025390625  MB
***credits***
Original dataset uses  1.040985107421875  MB
dataset uses  0.8675079345703125  MB after changes
memory saved = 0.1734771728515625  MB
***keywords***
Original dataset uses  0.7084197998046875  MB
dataset uses  1.4718647003173828  MB after changes
memory saved = -0.7634449005126953  MB
***links***
Original dataset uses  1.0493850708007812  MB
dataset uses  0.5247535705566406  MB after changes
memory saved = 0.5246315002441406  MB
***ratings***
Original dataset uses  794.1983947753906  MB
dataset uses  397.09925842285156  MB after changes
memory saved = 397.09913635253906  MB


In [6]:
ratings_pd

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
...,...,...,...,...
26024284,270896,58559,5.0,1257031564
26024285,270896,60069,5.0,1257032032
26024286,270896,63082,4.5,1257031764
26024287,270896,64957,4.5,1257033990


In [7]:
credits_pd

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [8]:
keywords_pd

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [ ]:
movies_metadata_pd

In [14]:
movies_metadata_pd.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [10]:
links_pd

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
45838,176269,6209470,439050.0
45839,176271,2028550,111109.0
45840,176273,303758,67758.0
45841,176275,8536,227506.0
